In [ ]:
!pip install transformers
!pip install accelerate -U
!pip install evaluate
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import torch
import re
import evaluate

import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
data_path = 'preprocessed_text.csv'
test_path = 'preprocessed_submission.csv'

In [ ]:
df_train = pd.read_csv(data_path, on_bad_lines='skip')
df_train

,Unnamed: 0,bug id,summary,severity,filtered_text,classification
0,0,365569,Remove workaround from bug 297227,normal,remove workaround bug,3
1,1,365578,Print Preview crashes on any URL in gtk2 builds,critical,print preview crash url gtk build,6
2,2,365582,Lines are not showing in table,major,line show table,4
3,3,365584,Firefox render ÛÏsimplified ArabicÛ font fa...,normal,firefox render simplify arabic font face incor...,3
4,4,365597,Crash [@ nsINodeInfo::NodeInfoManager],critical,crash ns node info node info manager,6
...,...,...,...,...,...,...
159993,159993,1143381,block elements with height after float left or...,normal,block elements height float leave right corrup...,3
159994,159994,1143392,typing in google translate will send reset inp...,normal,type google translate send reset input method ...,3
159995,159995,1143394,[gstreamer] Nightly instantly crashes on Youtu...,critical,gstreamer nightly instantly crash youtube linu...,6
159996,159996,1143395,Right click on Flash object with accessibility...,critical,right click flash object accessibility active ...,6


In [ ]:
df_test = pd.read_csv(test_path, on_bad_lines='skip')
df_test

,Unnamed: 0,bug id,summary,filtered_text,pred
0,0,1143402,Firefox claims to be not the default browser w...,firefox claim default browser update alter...,3
1,1,1143405,Background of html and body element are not ap...,background html body element apply correctly i...,3
2,2,1143409,Mouse input breaks after using window.showModa...,mouse input break use window show modal dial...,3
3,3,1143411,Build failure with next freetype version/curre...,build failure next freetype version current ...,3
4,4,1143417,HTML element is not treated as root inside for...,html element treat root inside foreign object,3
...,...,...,...,...,...
86089,86089,1426166,Crash in bool IsAboutToBeFinalizedInternal<T>,crash bool finalize internal,3
86090,86090,1426171,Potential crash if GraphRate is greater than 4...,potential crash graph rate greater k hz webrt...,3
86091,86091,1426173,Crash in <name omitted> | decltype JS::Dispatc...,crash name omit decltype js dispatch t...,3
86092,86092,1426174,Crash in xul.dll@0x28145fa | xul.dll@0x3c748ff...,crash xul dll x fa xul dll x c ff ...,3


In [ ]:
label_encoder = LabelEncoder()

y_numeric = label_encoder.fit_transform(df_train['severity'])
X_train, X_test, y_train, y_test = train_test_split(df_train, y_numeric, test_size=0.2, random_state=42, stratify=y_numeric)
y_numeric = label_encoder.fit_transform(X_train['severity'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_numeric)

In [ ]:
dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

{'blocker': 0,
 'critical': 1,
 'enhancement': 2,
 'major': 3,
 'minor': 4,
 'normal': 5,
 'trivial': 6}

In [ ]:
category_list = [words for words in df_train['severity'].to_list()]
category_list = list(set(category_list))[::-1]
label2id = {item: index for index, item in enumerate(category_list)}
id2label = {index: item for index, item in enumerate(category_list)}

In [ ]:
label2id

{'normal': 0,
 'blocker': 1,
 'minor': 2,
 'critical': 3,
 'enhancement': 4,
 'major': 5,
 'trivial': 6}

In [ ]:
id2label

{0: 'normal',
 1: 'blocker',
 2: 'minor',
 3: 'critical',
 4: 'enhancement',
 5: 'major',
 6: 'trivial'}

In [ ]:
X_train['labels'] = X_train['severity'].replace(label2id)
X_test['labels'] = X_test['severity'].replace(label2id)
#X_val['labels'] = X_val['severity'].replace(label2id)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(data_frame):
    return tokenizer(data_frame["filtered_text"], truncation=True)

In [ ]:
train_dataset = Dataset.from_pandas(X_train[['filtered_text', 'labels']])
test_dataset = Dataset.from_pandas(X_test[['filtered_text', 'labels']])

tokenized_train_df = train_dataset.map(preprocess_function, batched=True)
tokenized_test_df = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/102390 [00:00<?, ? examples/s]

Map:   0%|          | 0/31998 [00:00<?, ? examples/s]

In [ ]:
val_dataset = Dataset.from_pandas(x_val)

tokenized_val_df = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/25600 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
precision = evaluate.load("precision") #for macro precision
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return precision.compute(predictions=predictions, references=labels, average = 'macro')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=len(X_train['labels'].value_counts()), id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir= '/content/drive/MyDrive/sabanci_sunum/',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_df,
    eval_dataset=tokenized_test_df,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train() #with this preprocessor

Epoch,Training Loss,Validation Loss,Precision
1,0.539400,0.528414,0.332103
2,0.494800,0.529623,0.560136
3,0.415500,0.547342,0.695080


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=19200, training_loss=0.4949422562122345, metrics={'train_runtime': 1547.8914, 'train_samples_per_second': 198.444, 'train_steps_per_second': 12.404, 'total_flos': 2219559280134564.0, 'train_loss': 0.4949422562122345, 'epoch': 3.0})

In [ ]:
df_test = df_test.replace('N/A'," ")

In [ ]:
submission_dataset = Dataset.from_pandas(df_test[['filtered_text']])
tokenized_submission_df = submission_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/86094 [00:00<?, ? examples/s]

In [ ]:
result = trainer.predict(tokenized_submission_df)
result

PredictionOutput(predictions=array([[ 3.0221803 , -2.3315885 , -0.27449262, ..., -0.29899165,
         0.44630373, -1.1332802 ],
       [ 3.4492555 , -2.787294  ,  0.343203  , ..., -1.0462797 ,
         0.40871945, -0.8334625 ],
       [ 3.093691  , -1.9417846 , -0.4066331 , ..., -2.1828926 ,
         1.4493216 , -1.774365  ],
       ...,
       [ 1.9562358 , -0.9886433 , -3.1178112 , ..., -3.1043785 ,
        -1.1164244 , -3.939883  ],
       [ 1.5897529 , -0.95537674, -3.0223804 , ..., -3.1014924 ,
        -1.1782285 , -3.713039  ],
       [ 3.6944695 , -1.129734  , -0.84431046, ..., -1.3455794 ,
        -0.03155104, -1.6302204 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 81.6753, 'test_samples_per_second': 1054.1, 'test_steps_per_second': 65.883})

In [ ]:
result.predictions.shape

(86094, 7)

In [ ]:
df_test['severity'] = ""

In [ ]:
i = 0
for logits in result.predictions:
  df_test.loc[i,'severity'] = model.config.id2label[logits.argmax().item()]
  i = i + 1

In [ ]:
df_test

,Unnamed: 0,bug id,summary,filtered_text,pred,severity
0,0,1143402,Firefox claims to be not the default browser w...,firefox claim default browser update alter...,3,normal
1,1,1143405,Background of html and body element are not ap...,background html body element apply correctly i...,3,normal
2,2,1143409,Mouse input breaks after using window.showModa...,mouse input break use window show modal dial...,3,normal
3,3,1143411,Build failure with next freetype version/curre...,build failure next freetype version current ...,3,normal
4,4,1143417,HTML element is not treated as root inside for...,html element treat root inside foreign object,3,normal
...,...,...,...,...,...,...
86089,86089,1426166,Crash in bool IsAboutToBeFinalizedInternal<T>,crash bool finalize internal,3,critical
86090,86090,1426171,Potential crash if GraphRate is greater than 4...,potential crash graph rate greater k hz webrt...,3,normal
86091,86091,1426173,Crash in <name omitted> | decltype JS::Dispatc...,crash name omit decltype js dispatch t...,3,critical
86092,86092,1426174,Crash in xul.dll@0x28145fa | xul.dll@0x3c748ff...,crash xul dll x fa xul dll x c ff ...,3,critical


In [ ]:
df_test = df_test.rename(columns={'bug id': 'bug_id'})
df_test.set_index("bug_id", inplace=True)

In [ ]:
df_test = df_test[['severity']]

In [ ]:
df_test.to_csv("bert.csv")